In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model = model.to("cuda")

In [2]:
raw_prompt = """You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.
Question: What did the study reveal about the role of external calcium concentration in the action potential and contraction recovery time of crayfish muscle fibers?

Options:
A. The study found that increasing external calcium concentration had no effect on the action potential or contraction recovery time in crayfish muscle fibers.
B. The research concluded that magnesium ions play a more significant role than calcium in the action potential and contraction recovery of crayfish muscle fibers.
C. The investigation revealed that external calcium concentration only affects the resting potential of crayfish muscle fibers, not the action potential or contraction recovery.
D. The study investigated how changes in external calcium concentration affect the action potential and contraction recovery time in crayfish muscle fibers, revealing that calcium entry through TTS membranes is crucial for excitation-contraction coupling.

Between A, B, C and D the best option is"""

In [3]:
raw_prompt

'You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.\nQuestion: What did the study reveal about the role of external calcium concentration in the action potential and contraction recovery time of crayfish muscle fibers?\n\nOptions:\nA. The study found that increasing external calcium concentration had no effect on the action potential or contraction recovery time in crayfish muscle fibers.\nB. The research concluded that magnesium ions play a more significant role than calcium in the action potential and contraction recovery of crayfish muscle fibers.\nC. The investigation revealed that external calcium concentration only affects the resting potential of crayfish muscle fibers, not the action potential or contraction recovery.\nD. The study investigated how changes in external calcium concentration affect the action potential and contraction recovery time in crayfish muscle fibers, revealing that c

In [4]:
tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer(
    raw_prompt,
    return_tensors="pt",
).to("cuda")

tokenized

{'input_ids': tensor([[128000,   2675,    527,    459,   6335,    304,   5361,  63726,   4860,
             13,   4718,   3465,    374,    311,   3373,    279,   1888,   4320,
            505,    279,   2728,   2671,   3196,    389,    279,   3984,   2317,
            627,  14924,     25,   3639,   1550,    279,   4007,  16805,    922,
            279,   3560,    315,   9434,  35719,  20545,    304,    279,   1957,
           4754,    323,  71895,  13654,    892,    315,  90016,  18668,  16124,
          49774,   1980,   3883,    512,     32,     13,    578,   4007,   1766,
            430,   7859,   9434,  35719,  20545,   1047,    912,   2515,    389,
            279,   1957,   4754,    477,  71895,  13654,    892,    304,  90016,
          18668,  16124,  49774,    627,     33,     13,    578,   3495,  20536,
            430,  61933,  65125,   1514,    264,    810,   5199,   3560,   1109,
          35719,    304,    279,   1957,   4754,    323,  71895,  13654,    315,
          9001

In [5]:
tokenizer.decode(tokenized["input_ids"][0])

'<|begin_of_text|>You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.\nQuestion: What did the study reveal about the role of external calcium concentration in the action potential and contraction recovery time of crayfish muscle fibers?\n\nOptions:\nA. The study found that increasing external calcium concentration had no effect on the action potential or contraction recovery time in crayfish muscle fibers.\nB. The research concluded that magnesium ions play a more significant role than calcium in the action potential and contraction recovery of crayfish muscle fibers.\nC. The investigation revealed that external calcium concentration only affects the resting potential of crayfish muscle fibers, not the action potential or contraction recovery.\nD. The study investigated how changes in external calcium concentration affect the action potential and contraction recovery time in crayfish muscle fibers,

In [6]:
with torch.no_grad():
    model.eval()
    # Generate the output
    out = model(
        **tokenized,
    )


In [7]:
out[0][-1]

tensor([[ 7.0544,  9.0268, 13.3232,  ..., -3.7595, -3.7596, -3.7596],
        [12.8574, 11.2714,  9.1116,  ...,  0.0396,  0.0400,  0.0398],
        [10.5716,  5.0304,  6.1321,  ..., -0.3093, -0.3089, -0.3093],
        ...,
        [ 9.2915,  9.5077,  5.4119,  ...,  1.1720,  1.1720,  1.1726],
        [12.4992, 11.2127,  8.8542,  ...,  1.7499,  1.7493,  1.7499],
        [12.8659, 13.4886,  9.0342,  ...,  1.4060,  1.4054,  1.4061]],
       device='cuda:0')

In [8]:
id = torch.argmax(out.logits[0][-1])

In [9]:
id

tensor(423, device='cuda:0')

In [10]:
tokenizer.decode(id)

' D'

In [13]:
generate = model.generate(
    input_ids=tokenized["input_ids"],
    attention_mask=tokenized["attention_mask"],
    do_sample=False,
    logits_processor=None,
)

/home/ubuntu/miniconda3/envs/faiss39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/faiss39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
print(generate)

tensor([[128000,   2675,    527,    459,   6335,    304,   5361,  63726,   4860,
             13,   4718,   3465,    374,    311,   3373,    279,   1888,   4320,
            505,    279,   2728,   2671,   3196,    389,    279,   3984,   2317,
            627,  14924,     25,   3639,   1550,    279,   4007,  16805,    922,
            279,   3560,    315,   9434,  35719,  20545,    304,    279,   1957,
           4754,    323,  71895,  13654,    892,    315,  90016,  18668,  16124,
          49774,   1980,   3883,    512,     32,     13,    578,   4007,   1766,
            430,   7859,   9434,  35719,  20545,   1047,    912,   2515,    389,
            279,   1957,   4754,    477,  71895,  13654,    892,    304,  90016,
          18668,  16124,  49774,    627,     33,     13,    578,   3495,  20536,
            430,  61933,  65125,   1514,    264,    810,   5199,   3560,   1109,
          35719,    304,    279,   1957,   4754,    323,  71895,  13654,    315,
          90016,  18668,  16

In [14]:
tokenizer.decode(generate[0])

'<|begin_of_text|>You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.\nQuestion: What did the study reveal about the role of external calcium concentration in the action potential and contraction recovery time of crayfish muscle fibers?\n\nOptions:\nA. The study found that increasing external calcium concentration had no effect on the action potential or contraction recovery time in crayfish muscle fibers.\nB. The research concluded that magnesium ions play a more significant role than calcium in the action potential and contraction recovery of crayfish muscle fibers.\nC. The investigation revealed that external calcium concentration only affects the resting potential of crayfish muscle fibers, not the action potential or contraction recovery.\nD. The study investigated how changes in external calcium concentration affect the action potential and contraction recovery time in crayfish muscle fibers,